In [116]:
import sys
sys.path.append('../build/mass_spring')

from mass_spring import *

from pythreejs import *

In [117]:
mss = MassSpringSystem3d()
mss.gravity = (0,-9.81,0)

height = 3
width = 3

objects=[[]]

for i in range(-1, width):
    objects[0].append(mss.Add(Fix((i,0,0)))) # top fix

for i in range(1, height):
    objects.append([mss.Add(Fix((-1,0,i)))]) # left fix
    for j in range(width):
        mass = mss.Add(Mass(i+j, (j, 0, i)))
        objects[i].append(mass)
        mss.Add(Spring(1, j*200, (objects[i][-2], objects[i][-1]))) # connect left
        mss.Add(Spring(1, i*200, (objects[i][-1], objects[i-1][j + 1]))) # connect up
    objects[i].append(mss.Add(Fix((width,0,i)))) # right fix
    mss.Add(Spring(1, i*200, (objects[i][-2], objects[i][-1])))

for i in range(-1, width):
    objects[-1].append(mss.Add(Fix((i,0,height)))) # bottom fix
    mss.Add(Spring(1, i*200, (objects[-1][-1], objects[-1][i + 1]))) # connect up

In [118]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=(m.pos[0], m.pos[1], m.pos[2])))

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=(f.pos[0], f.pos[1], f.pos[2])))

springpos = []
for s in mss.springs:
    pA = mss[s.connections[0]].pos
    pB = mss[s.connections[1]].pos
    springpos.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [119]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [120]:
from time import sleep
for i in range(10000):
    Simulate (mss, 0.02, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connections[0]].pos
        pB = mss[s.connections[1]].pos
        springpos.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)

KeyboardInterrupt: 